___
<a name="setup"></a>
## Dataset creation

In [1]:
from config import *
import numpy as np
import pandas as pd

import re
import os
import time
from datetime import datetime, date, timedelta

import tweepy

import preprocessor as p





#### Twitter API credentials setup

In [2]:
auth=tweepy.OAuthHandler(TWITTER_CONSUMER_KEY,TWITTER_CONSUMER_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)
api=tweepy.API(auth)


___
<a name="api"></a>
## Data Extraction with Twitter API and Text Pre-Processing

In [3]:
list_of_dates = []
today = date.today()
for i in range(-7,1):
    target_date = (today + timedelta(days=i)).strftime("%Y-%m-%d")
    list_of_dates.append(target_date)

In [4]:
list_of_dicts = []
search_term = 'covid19 covid'
num_tweets = 1000

In [5]:
def get_tweets(search_term = search_term, num_tweets = num_tweets):
    
    for end_date in list_of_dates:
        start_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d") 
        tweet_count = len(list_of_dicts)

        for tweet in tweepy.Cursor(api.search,
                                   q=f'{search_term} since:{start_date} until:{end_date}',
                                   lang = 'en',
                                   count = num_tweets,
                                   tweet_mode = 'extended').items(num_tweets):
            if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                if tweet.lang == "en":
                    tweet_dict = {}
                    tweet_dict['username'] = tweet.user.name
                    tweet_dict['location'] = tweet.user.location
                    tweet_dict['text'] = tweet.full_text
                    #tweet_dict['fav_count'] = tweet.favorite_count  
                    tweet_dict['hashtags'] = tweet.entities['hashtags']
                    tweet_dict['tweet_date'] = tweet.created_at
                    list_of_dicts.append(tweet_dict)
                    tweet_count +=1
                    print(f'Extracted tweet count = {tweet_count}')
                
        print(f'Completed extraction for {start_date} to {end_date}. Sleep for 15 mins')
        time.sleep(900)
        print('Ready to go again')

In [6]:
get_tweets()

Extracted tweet count = 1
Extracted tweet count = 2
Extracted tweet count = 3
Extracted tweet count = 4
Extracted tweet count = 5
Extracted tweet count = 6
Extracted tweet count = 7
Extracted tweet count = 8
Extracted tweet count = 9
Extracted tweet count = 10
Extracted tweet count = 11
Extracted tweet count = 12
Extracted tweet count = 13
Extracted tweet count = 14
Extracted tweet count = 15
Extracted tweet count = 16
Extracted tweet count = 17
Extracted tweet count = 18
Extracted tweet count = 19
Extracted tweet count = 20
Extracted tweet count = 21
Extracted tweet count = 22
Extracted tweet count = 23
Extracted tweet count = 24
Extracted tweet count = 25
Extracted tweet count = 26
Extracted tweet count = 27
Extracted tweet count = 28
Extracted tweet count = 29
Extracted tweet count = 30
Extracted tweet count = 31
Extracted tweet count = 32
Extracted tweet count = 33
Extracted tweet count = 34
Extracted tweet count = 35
Extracted tweet count = 36
Extracted tweet count = 37
Extracted 

In [ ]:
len(list_of_dicts)

28991

In [ ]:
tweets_df = pd.DataFrame(list_of_dicts)
tweets_df.sort_values(by='tweet_date').reset_index(drop = True)

,username,location,text,hashtags,tweet_date
0,Jus,UK,.“This bill is vital to address the reality th...,"[{'text': 'COVID19', 'indices': [218, 226]}, {...",2021-05-29 14:58:09
1,WORLD UPDATES #coronavirus disease 2019 (COVID...,United Kingdom,ENGLAND. COVID-19 Daily Deaths.\n\nAs from 14t...,"[{'text': 'coronavirus', 'indices': [128, 140]...",2021-05-29 14:58:21
2,Mashable Southeast Asia,Malaysia,Girl power.\n#MashableSEA #Science #Fetus #COV...,"[{'text': 'MashableSEA', 'indices': [12, 24]},...",2021-05-29 15:00:00
3,CarlisleCityCouncil,"Carlisle, Cumbria. UK",Join the millions already vaccinated. \n\nThe ...,"[{'text': 'COVID19', 'indices': [76, 84]}]",2021-05-29 15:00:02
4,GI Kids,NASPGHAN Foundation,"Children with chronic liver disorders, includi...","[{'text': 'hepatitis', 'indices': [49, 59]}, {...",2021-05-29 15:00:05
...,...,...,...,...,...
28986,Steve™,"San Francisco, CA","After COVID-19 successes, researchers push to ...","[{'text': 'covid19', 'indices': [112, 120]}]",2021-06-05 23:55:52
28987,Walle,,@Dad_of_Marine Btw 5165 of 1 percent reported ...,[],2021-06-05 23:57:30
28988,Opoyi,,#COVID19 has been here for more than one year ...,"[{'text': 'COVID19', 'indices': [0, 8]}, {'tex...",2021-06-05 23:57:35
28989,"Umbereen S. Nehal, MD, MPH",in the air,Mom had chronic pain. She was so stoic about i...,"[{'text': 'COVID19', 'indices': [71, 79]}]",2021-06-05 23:58:59


In [ ]:
def extract_hashtags(hashtag_list):
    
    s = "" 
    if not hashtag_list: 
        return s
    else:
        for dictionary in hashtag_list:
            s+= str(dictionary['text'].lower() + ',') 
        s = s[:-1] 
        return s

In [ ]:
# Extract hashtags
tweets_df['hashtags_extracted'] = tweets_df['hashtags'].apply(lambda x: extract_hashtags(x))
tweets_df.drop(columns = 'hashtags', inplace = True)

In [ ]:
tweets_df.head()

,username,location,text,tweet_date,hashtags_extracted
0,Habari Entertainment,Phoenix,Illinois COVID Update Today: IL reports 802 ca...,2021-05-29 23:59:58,covid19
1,Beverly,New York,Bombshell New study says Chinese scientists cr...,2021-05-29 23:57:46,"covid19,media,usa"
2,Boston Athlete Magazine,United States,Boston Athlete News - Today begins lifting of ...,2021-05-29 23:56:28,"cityofboston,boston,massachusetts,bostonathlet..."
3,vare ramankumar,,@MoHFW_INDIA @PrakashJavdekar @PMOIndia @drhar...,2021-05-29 23:56:07,
4,Tom,"Kildare, Ireland",Is 'Patient Su' Covid's Patient Zero? asks IAN...,2021-05-29 23:55:14,"covid19,wuhanlab"


In [ ]:

# len(tweets_df_final)

In [ ]:
extract_datetime = datetime.today().strftime('%Y%m%d_%H%M%S')

filename = 'dataset.csv'

tweets_df.drop_duplicates(inplace = True)

tweets_df.to_csv(filename, index = False)